<!-- DISABLE-FRONTMATTER-SECTIONS -->

# Speed Comparison

`Safetensors` is really fast. Let's compare it against `PyTorch` by loading [gpt2](https://huggingface.co/gpt2) weights. To run the [GPU benchmark](#gpu-benchmark), make sure your machine has GPU or you have selected `GPU runtime` if you are using Google Colab.

Before you begin, make sure you have all the necessary libraries installed:

In [1]:
!pip install safetensors huggingface_hub torch

Let's start by importing all the packages that will be used:

In [2]:
import os
import datetime
from huggingface_hub import hf_hub_download
from safetensors.torch import load_file
import torch

Download safetensors & torch weights for gpt2:

In [3]:
sf_filename = hf_hub_download("gpt2", filename="model.safetensors")
pt_filename = hf_hub_download("gpt2", filename="pytorch_model.bin")

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/548M [00:00<?, ?B/s]

### CPU benchmark

In [4]:
start_st = datetime.datetime.now()
weights = load_file(sf_filename, device="cpu")
load_time_st = datetime.datetime.now() - start_st
print(f"Loaded safetensors {load_time_st}")

start_pt = datetime.datetime.now()
weights = torch.load(pt_filename, map_location="cpu")
load_time_pt = datetime.datetime.now() - start_pt
print(f"Loaded pytorch {load_time_pt}")

print(f"on CPU, safetensors is faster than pytorch by: {load_time_pt/load_time_st:.1f} X")

Loaded safetensors 0:00:00.106408
Loaded pytorch 0:00:00.326319
on CPU, safetensors is faster than pytorch by: 3.1 X


This speedup is due to the fact that this library avoids unnecessary copies by mapping the file directly. It is actually possible to do on [pure pytorch](https://gist.github.com/Narsil/3edeec2669a5e94e4707aa0f901d2282).
The currently shown speedup was gotten on:
* OS: Ubuntu 18.04.6 LTS
* CPU: Intel(R) Xeon(R) CPU @ 2.00GHz

### GPU benchmark

In [5]:
# This is required because this feature hasn't been fully verified yet, but
# it's been tested on many different environments
os.environ["SAFETENSORS_FAST_GPU"] = "1"

# CUDA startup out of the measurement
torch.zeros((2, 2)).cuda()

start_st = datetime.datetime.now()
weights = load_file(sf_filename, device="cuda:0")
load_time_st = datetime.datetime.now() - start_st
print(f"Loaded safetensors {load_time_st}")

start_pt = datetime.datetime.now()
weights = torch.load(pt_filename, map_location="cuda:0")
load_time_pt = datetime.datetime.now() - start_pt
print(f"Loaded pytorch {load_time_pt}")

print(f"on GPU, safetensors is faster than pytorch by: {load_time_pt/load_time_st:.1f} X")

Loaded safetensors 0:00:00.179590
Loaded pytorch 0:00:00.505419
on GPU, safetensors is faster than pytorch by: 2.8 X


The speedup works because this library is able to skip unecessary CPU allocations. It is unfortunately not replicable in pure pytorch as far as we know. The library works by memory mapping the file, creating the tensor empty with pytorch and calling `cudaMemcpy` directly to move the tensor directly on the GPU.
The currently shown speedup was gotten on:
* OS: Ubuntu 18.04.6 LTS.
* GPU: Tesla T4
* Driver Version: 460.32.03
* CUDA Version: 11.2

In [6]:
!pip install -U llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 MB 21.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.9 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.16-cp312-cp312-linux_x86_64.whl size=4503216 sha256=fcfa35dc93180116e90a0b15ec766e6fa5ba70cb7f10e363c7247d1f34592684
  Stored in directory: /root/.cache/pip/wheels/90/82/ab/8784ee3fb99ddb07fd36a679ddbe63122cc07718f6c1eb3be8
Successfully built llama-cpp-python


In [7]:
# !pip install llama-cpp-python

from llama_cpp import Llama

llm = Llama.from_pretrained(
	repo_id="pahaadi/Qwen2.5-Coder-14B-houdini_vex_functions",
	filename="unsloth.Q8_0.gguf",
)


./unsloth.Q8_0.gguf:   0%|          | 0.00/15.7G [00:00<?, ?B/s]

llama_model_loader: loaded meta data with 27 key-value pairs and 579 tensors from /root/.cache/huggingface/hub/models--pahaadi--Qwen2.5-Coder-14B-houdini_vex_functions/snapshots/8c929f365a20b86a58f77ce87e8933ea228d680a/./unsloth.Q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = qwen2
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Qwen2.5 Coder 14b Instruct Bnb 4bit
llama_model_loader: - kv   3:                       general.organization str              = Unsloth
llama_model_loader: - kv   4:                           general.finetune str              = instruct-bnb-4bit
llama_model_loader: - kv   5:                           general.basename str              = qwen2.5-co

In [8]:
llm.create_chat_completion(
	messages = "No input example has been defined for this model task."
)

llama_perf_context_print:        load time =   40323.38 ms
llama_perf_context_print: prompt eval time =   40323.19 ms /    24 tokens ( 1680.13 ms per token,     0.60 tokens per second)
llama_perf_context_print:        eval time =  348271.38 ms /     9 runs   (38696.82 ms per token,     0.03 tokens per second)
llama_perf_context_print:       total time =  388663.63 ms /    33 tokens
llama_perf_context_print:    graphs reused =          7


{'id': 'chatcmpl-e1b9707b-99ac-4471-810d-8e32d4da24a5',
 'object': 'chat.completion',
 'created': 1768972180,
 'model': '/root/.cache/huggingface/hub/models--pahaadi--Qwen2.5-Coder-14B-houdini_vex_functions/snapshots/8c929f365a20b86a58f77ce87e8933ea228d680a/./unsloth.Q8_0.gguf',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': 'Hello! How can I assist you today?'},
   'logprobs': None,
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 24, 'completion_tokens': 9, 'total_tokens': 33}}